In [ ]:
pip install torch==2.2.0 transformers==4.40 torchmetrics einops accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.6/137.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 126.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Upgrade both to compatible versions (Torch 2.2.1 and torchvision 0.17.1, for example)
%pip install torch==2.2.1 torchvision==0.17.1 --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.6/755.6 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 43.8 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.2.0
    Uninstalling torch-2.2.0:
      Successfully uninstalled torch-2.2.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.21.0+cu124
    Uninstalling torchvision-0.21.0+cu124:
      Successfully uninstalled torchvision-0.21.0+cu124
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have transformers 4.40.0 which is incompatible.
torchaudio 2.6.0+cu124 requires torch==2.6.0, but you have torch 2.2.1 which is incompatible.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv, math, random, torch, tqdm, functools
import torch.nn as nn
from pathlib import Path
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torchmetrics.classification import F1Score

In [ ]:
DATA_PATH   = "/content/drive/MyDrive/DLProj/Dyadic_PELD.tsv"           # ⬅️  your single TSV file
VAD_PATH    = "/content/drive/MyDrive/DLProj/NRC-VAD-Lexicon.txt"
TRAIN_PCT   = 0.80
VAL_PCT     = 0.10      # test share is 1-train-val
EPOCHS      = 5
BATCH_SIZE  = 8
RANDOM_SEED = 42
BERT_NAME   = "bert-base-uncased"

In [ ]:

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_vad_lexicon(path, normalise=True):
    lex = {}
    with open(path) as f:
        for tok, v, a, d in csv.reader(f, delimiter='\t'):
            if tok.startswith('#') or tok == '':
                continue
            vec = torch.tensor([float(v), float(a), float(d)])
            if normalise:
                vec = (vec - 0.5) * 2
            lex[tok.lower()] = vec
    return lex

LABEL2ID = {'NEUTRAL':0,'JOY':1,'SURPRISE':2,'ANGER':3,
            'FEAR':4,'SADNESS':5,'DISGUST':6}

In [ ]:
# top of the notebook
LABEL2ID = {'NEUTRAL':0,'JOY':1,'SURPRISE':2,'ANGER':3,
            'FEAR':4,'SADNESS':5,'DISGUST':6}

ALIASES = {           # maps raw strings → canonical PaMT label
    'POSITIVE':'JOY',
    'NEGATIVE':'SADNESS',
    'NEUTRAL':'NEUTRAL',   # already there
    'HAPPINESS':'JOY',
    '😃':'JOY',
    # add more if your file has them …
}

In [ ]:
class PELDDataset(Dataset):
    """
    Loads one PELD-style TSV row per dialog, tokenises three utterances,
    attaches NRC-VAD vectors to every token, and returns tensors that the
    HTN-plus-GRU model expects.

    Expected file columns (tab-separated):
        Speaker_1 Speaker_2 Personality Utterance_1 Utterance_2 Utterance_3
        Emotion_1 Emotion_2 Emotion_3  (… any extra cols are ignored)
    """
    def __init__(self, tsv_path: str, tokenizer, vad_lex: dict, max_len: int = 64):
        self.rows   = Path(tsv_path).read_text().strip().split('\n')[1:]   # skip header
        self.tok    = tokenizer
        self.lex    = vad_lex
        self.max_len= max_len

    # ---------- helpers ----------
    def _avg_vad(self, vads: torch.Tensor) -> torch.Tensor:
        """Average VAD over tokens that have a non-zero lexicon entry."""
        mask = vads.abs().sum(-1) > 0          # (L,) True where token had a hit
        return vads[mask].mean(0) if mask.any() else torch.zeros(3)

    def _clean_personality(self, raw: str) -> torch.Tensor:
        """
        '[0.64, 0.37, 0.51, 0.42, 0.19]'  ->  tensor([0.64, 0.37, 0.51, 0.42, 0.19])
        """
        raw  = raw.strip().lstrip('[').rstrip(']')
        vals = [float(x) for x in raw.replace(' ', '').split(',') if x]
        if len(vals) != 5:
            raise ValueError(f"Expected 5 OCEAN numbers, got {vals}")
        return torch.tensor(vals)

    # ---------- standard Dataset API ----------
    def __len__(self) -> int:
        return len(self.rows)

    def __getitem__(self, idx: int) -> dict:
        cells   = self.rows[idx].split('\t')

        persona = self._clean_personality(cells[2])
        utts    = cells[3:6]
        emos    = cells[6:9]
                               # Emotion_1..3

        tok_ids, masks, tok_vads, utt_vads = [], [], [], []
        for u in utts:
            enc   = self.tok(u,
                             truncation=True,
                             padding='max_length',
                             max_length=self.max_len,
                             return_tensors='pt')
            ids   = enc.input_ids.squeeze()                  # (L,)
            mask  = enc.attention_mask.squeeze()
            toks  = self.tok.convert_ids_to_tokens(ids)

            vads  = torch.stack([self.lex.get(t.lower(), torch.zeros(3))
                                 for t in toks])             # (L,3)

            tok_ids.append(ids)
            masks.append(mask)
            tok_vads.append(vads)
            utt_vads.append(self._avg_vad(vads))             # (3,)

        prev_mood = utt_vads[0]
        vad_gold  = utt_vads[2]

        raw = emos[2].strip().upper()
        canon = ALIASES.get(raw, raw)
        emo_label = torch.tensor(LABEL2ID[canon])

        return {
            "input_ids": torch.stack(tok_ids),
            "attn_mask": torch.stack(masks),
            "tok_vad"  : torch.stack(tok_vads),
            "persona"  : persona,
            "prev_mood": prev_mood,
            "vad_gold" : vad_gold,
            "emo_label": emo_label
        }

In [ ]:
def collate(batch):
    return {k: torch.stack([b[k] for b in batch]) for k in batch[0]}


In [ ]:
class AffectAttn(nn.Module):
    def __init__(self, hid=768):
        super().__init__()
        self.q = nn.Linear(hid, 3)
        self.scale = hid ** -0.5
    def forward(self, h, tok_vad, prev_mood):
        att = (self.q(h) * tok_vad).sum(-1) * self.scale
        att = att.masked_fill(tok_vad.sum(-1).eq(0), -1e4)
        w   = att.softmax(-1).unsqueeze(-1)
        vad_hat = (w * tok_vad).sum(1)
        return vad_hat - prev_mood.view(-1,3)


In [ ]:
class ContextTF(nn.Module):
    def __init__(self, d=768, L=2, H=8):
        super().__init__()
        layer = nn.TransformerEncoderLayer(d, H, 4*d, batch_first=True)
        self.enc = nn.TransformerEncoder(layer, L)
    def forward(self, x): return self.enc(x)

In [ ]:
class MoodGRU(nn.Module):
    def __init__(self, ctx_dim=768, h=256):
        super().__init__()
        self.gru = nn.GRU(ctx_dim, h, batch_first=True)
        self.fc  = nn.Linear(h+5, 3)
    def forward(self, ctx_seq, persona, prev):
        _, h = self.gru(ctx_seq)
        delta = self.fc(torch.cat([h.squeeze(0), persona], -1))
        return torch.tanh(prev + delta)

In [ ]:
class EmotionHead(nn.Module):
    def __init__(self, ctx_dim=768, n_em=7):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(ctx_dim+3+5, 256), nn.GELU(), nn.Dropout(0.1),
            nn.Linear(256, n_em))
    def forward(self, cls_last, mood, persona):
        return self.net(torch.cat([cls_last, mood, persona], -1))


In [ ]:
class PaMT(nn.Module):
    def __init__(self):
        super().__init__()
        self.bert = AutoModel.from_pretrained(BERT_NAME)
        self.aff  = AffectAttn(self.bert.config.hidden_size)
        self.ctx  = ContextTF(self.bert.config.hidden_size)
        self.mood = MoodGRU(self.bert.config.hidden_size)
        self.em   = EmotionHead(self.bert.config.hidden_size)
    def forward(self, batch):
        B,T,L = batch["input_ids"].shape
        ids   = batch["input_ids"].view(B*T, L).to(DEVICE)
        mask  = batch["attn_mask"].view(B*T, L).to(DEVICE)

        out   = self.bert(ids, attention_mask=mask).last_hidden_state
        prev_rep = batch["prev_mood"].to(DEVICE).repeat_interleave(T, dim=0)
        delta = self.aff(out,
                 batch["tok_vad"].view(B*T, L, 3).to(DEVICE),
                 prev_rep)
        delta = delta.view(B,T,3).sum(1)

        cls_seq = out[:,0].view(B,T,-1)
        ctx_seq = self.ctx(cls_seq)
        cls_last= ctx_seq[:,-1]

        mood = self.mood(ctx_seq,
                         batch["persona"].to(DEVICE),
                         batch["prev_mood"].to(DEVICE))
        logits= self.em(cls_last, mood, batch["persona"].to(DEVICE))
        return mood, logits


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha, gamma=2):
        super().__init__(); self.a, self.g = alpha, gamma
    def forward(self, logits, target):
        ce = nn.functional.cross_entropy(logits, target, reduction='none')
        pt = torch.exp(-ce)
        at = self.a.gather(0,target)
        return ((1-pt)**self.g * at * ce).mean()


In [ ]:
def split_ds(ds, p_train, p_val, seed):
    n = len(ds)
    n_train = int(n*p_train)
    n_val   = int(n*p_val)
    n_test  = n - n_train - n_val
    g = torch.Generator().manual_seed(seed)
    return random_split(ds, [n_train,n_val,n_test], generator=g)

In [ ]:
tok = AutoTokenizer.from_pretrained(BERT_NAME)
vad = load_vad_lexicon(VAD_PATH)
full_ds   = PELDDataset(DATA_PATH, tok, vad)
train_ds, val_ds, test_ds = split_ds(full_ds, TRAIN_PCT, VAL_PCT, RANDOM_SEED)

train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate)
val_dl   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)
test_dl  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate)

model = PaMT().to(DEVICE)
# class-imbalance α
labels_all = torch.tensor([b["emo_label"] for b in full_ds])
freq = torch.bincount(labels_all, minlength=7).float()
alpha = (1./freq).div((1./freq).sum()).to(DEVICE)
loss_mood = nn.MSELoss()
loss_emo  = FocalLoss(alpha)

opt   = torch.optim.AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)
sched = get_linear_schedule_with_warmup(
            opt, int(0.1*EPOCHS*len(train_dl)), EPOCHS*len(train_dl))
f1mac = F1Score(task="multiclass",
                num_classes=7,
                average="macro").to(DEVICE)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
for ep in range(1, EPOCHS+1):
    # ---- training ----
    model.train(); tot=0
    for batch in tqdm.tqdm(train_dl, desc=f"Epoch {ep}"):
        mood_hat, logits = model(batch)
        lm = loss_mood(mood_hat, batch["vad_gold"].to(DEVICE))
        le = loss_emo(logits, batch["emo_label"].to(DEVICE))
        loss = lm + 0.5*le
        loss.backward(); nn.utils.clip_grad_norm_(model.parameters(),1.0)
        opt.step(); sched.step(); opt.zero_grad()
        tot += loss.item()
    print(f"Epoch {ep}: train-loss {tot/len(train_dl):.4f}")

    # ---- validation ----
    model.eval(); f1mac.reset()
    with torch.no_grad():
        for batch in val_dl:
            _, logits = model(batch)
            f1mac.update(logits, batch["emo_label"].to(DEVICE))
    print(f"Epoch {ep}: val macro-F1 {f1mac.compute():.3f}")

Epoch 1: 100%|██████████| 651/651 [01:45<00:00,  6.18it/s]


Epoch 1: train-loss 0.1511
Epoch 1: val macro-F1 0.134


Epoch 2: 100%|██████████| 651/651 [01:47<00:00,  6.05it/s]


Epoch 2: train-loss 0.0907
Epoch 2: val macro-F1 0.287


Epoch 3: 100%|██████████| 651/651 [01:47<00:00,  6.03it/s]


Epoch 3: train-loss 0.0595
Epoch 3: val macro-F1 0.328


Epoch 4: 100%|██████████| 651/651 [01:47<00:00,  6.04it/s]


Epoch 4: train-loss 0.0434
Epoch 4: val macro-F1 0.339


Epoch 5: 100%|██████████| 651/651 [01:48<00:00,  6.03it/s]


Epoch 5: train-loss 0.0330
Epoch 5: val macro-F1 0.343


In [ ]:
model.eval(); f1mac.reset()
with torch.no_grad():
    for batch in test_dl:
        _, logits = model(batch)
        f1mac.update(logits, batch["emo_label"].to(DEVICE))
print("TEST macro-F1:", round(f1mac.compute().item(), 3))

TEST macro-F1: 0.368


In [ ]:
from torchmetrics.classification import F1Score
import torch

# ── initialise two metrics ──────────────────────────────────────────
f1_per_class = F1Score(task="multiclass",
                       num_classes=7,
                       average=None)    # returns a tensor(7,)
f1_macro     = F1Score(task="multiclass",
                       num_classes=7,
                       average="macro")

f1_per_class = f1_per_class.to(DEVICE)
f1_macro     = f1_macro.to(DEVICE)

# ── during evaluation (val or test) ─────────────────────────────────
model.eval()
f1_per_class.reset(); f1_macro.reset()

with torch.no_grad():
    for batch in test_dl:                      # val_dl or test_dl
        logits = model(batch)[1]                  # mood_hat, logits
        labels = batch["emo_label"].to(DEVICE)
        f1_per_class.update(logits, labels)
        f1_macro.update(logits, labels)

# ── get the results ────────────────────────────────────────────────
per_class_f1 = f1_per_class.compute().cpu().tolist()   # list of 7 floats
macro_f1     = f1_macro.compute().item()

print("Per-class F1:", per_class_f1)
print("Macro-F1   :", round(macro_f1, 3))


Per-class F1: [0.5505882501602173, 0.5502183437347412, 0.46043166518211365, 0.41999998688697815, 0.2522522509098053, 0.34567901492118835, 0.0]
Macro-F1   : 0.368


In [ ]:
ID2LABEL = {v: k for k, v in LABEL2ID.items()}

In [ ]:
per_class_f1 = f1_per_class.compute().cpu().tolist()   # length 7
macro_f1     = f1_macro.compute().item()

# nice, labelled print-out
print("─── Per-class F1 ─────────────────────────")
for cls_id in range(len(per_class_f1)):
    label = ID2LABEL[cls_id]          # e.g. 0 → 'NEUTRAL'
    print(f"{label:<9s}: {per_class_f1[cls_id]:.3f}")
print("─────────────────────────────────────────")
print(f"Macro-F1 : {macro_f1:.3f}")

─── Per-class F1 ─────────────────────────
NEUTRAL  : 0.551
JOY      : 0.550
SURPRISE : 0.460
ANGER    : 0.420
FEAR     : 0.252
SADNESS  : 0.346
DISGUST  : 0.000
─────────────────────────────────────────
Macro-F1 : 0.368


In [ ]:
from torchmetrics.classification import F1Score, Accuracy

# ── metric objects (create ONCE, before training/validation loops) ─────────
f1_per_class = F1Score(task="multiclass",
                       num_classes=7,
                       average=None).to(DEVICE)   # tensor(7,) – per-class
f1_macro     = F1Score(task="multiclass",
                       num_classes=7,
                       average="macro").to(DEVICE)

acc_overall  = Accuracy(task="multiclass",
                        num_classes=7,
                        top_k=1).to(DEVICE)        # standard accuracy
# ── validation / test loop ─────────────────────────────────────────────────
model.eval()
for m in (f1_per_class, f1_macro, acc_overall):
    m.reset()

with torch.no_grad():
    for batch in test_dl:             # or test_dl
        logits = model(batch)[1]     # mood_hat, logits
        labels = batch["emo_label"].to(DEVICE)

        # update all metrics
        f1_per_class.update(logits, labels)
        f1_macro.update(logits, labels)
        acc_overall.update(logits, labels)

# ── report ────────────────────────────────────────────────────────────────
per_cls  = f1_per_class.compute().cpu().tolist()
macro_f1 = f1_macro.compute().item()
accuracy = acc_overall.compute().item()

print("─── Validation metrics ───")
for i, f1 in enumerate(per_cls):
    print(f"{ID2LABEL[i]:<9s} F1: {f1:.3f}")
print("Macro-F1 :", round(macro_f1, 3))
print("Accuracy :", round(accuracy, 3))


─── Validation metrics ───
NEUTRAL   F1: 0.551
JOY       F1: 0.550
SURPRISE  F1: 0.460
ANGER     F1: 0.420
FEAR      F1: 0.252
SADNESS   F1: 0.346
DISGUST   F1: 0.000
Macro-F1 : 0.368
Accuracy : 0.455


In [ ]:
from google.colab import drive
drive.mount('/content/drive')        # already mounted → no harm

RUN_DIR = "/content/drive/MyDrive/PaMT-run-01"
!mkdir -p "$RUN_DIR"

# 1️⃣  save model weights
torch.save(model.state_dict(), f"{RUN_DIR}/pamt_state.pt")

# 2️⃣  save tokenizer (this *does* have save_pretrained)
tok.save_pretrained(RUN_DIR)

# 3️⃣  misc config you’ll want later
import json, pkg_resources, os
extras = {
    "bert_name" : BERT_NAME,
    "label2id"  : LABEL2ID,
    "aliases"   : ALIASES,
    "max_len"   : 64,
}
json.dump(extras, open(f"{RUN_DIR}/extras.json","w"), indent=2)

# 4️⃣  optional: freeze environment
!pip freeze > "$RUN_DIR/requirements.txt"

print("Saved to", RUN_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-25-736b75f3274d>:14: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import json, pkg_resources, os


Saved to /content/drive/MyDrive/PaMT-run-01


In [ ]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 961.5/961.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 111.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninst

In [ ]:
# Step 1: Import required libraries
import pandas as pd

# Step 3: Read the uploaded .tsv file
# Replace 'your_file.tsv' with the actual filename after upload
df = pd.read_csv('/content/drive/MyDrive/DLProj/Dyadic_PELD.tsv', sep='\t')

# Optional: Display first few rows
df.head()


,Speaker_1,Speaker_2,Personality,Utterance_1,Utterance_2,Utterance_3,Emotion_1,Emotion_2,Emotion_3,Sentiment_1,Sentiment_2,Sentiment_3
0,Chandler,The Interviewer,"[0.648, 0.375, 0.386, 0.58, 0.477]",also I was the point person on my company s tr...,You must ve had your hands full.,That I did. That I did.,neutral,neutral,neutral,neutral,neutral,neutral
1,Chandler,The Interviewer,"[0.648, 0.375, 0.386, 0.58, 0.477]",That I did. That I did.,So let s talk a little bit about your duties.,My duties? All right.,neutral,neutral,surprise,neutral,neutral,positive
2,Chandler,The Interviewer,"[0.648, 0.375, 0.386, 0.58, 0.477]",My duties? All right.,"Now you ll be heading a whole division, so you...",I see.,surprise,neutral,neutral,positive,neutral,neutral
3,Chandler,The Interviewer,"[0.648, 0.375, 0.386, 0.58, 0.477]",I see.,But there ll be perhaps 30 people under you so...,Good to know.,neutral,neutral,neutral,neutral,neutral,neutral
4,Chandler,The Interviewer,"[0.648, 0.375, 0.386, 0.58, 0.477]",Good to know.,We can go into detail,No don t I beg of you!,neutral,neutral,fear,neutral,neutral,negative


In [ ]:
# Group by 'Emotion_3' and count instances
emotion_counts = df['Emotion_3'].value_counts()

# Display the counts
print(emotion_counts)


Emotion_3
neutral     2771
joy         1123
anger        858
surprise     634
sadness      493
fear         487
disgust      144
Name: count, dtype: int64
